# call utility functions to get the analysis file

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import dhs_util
from dhs_util import *

os.chdir('/Users/yingli/Development/TopicsInDataScience/')
df = pd.read_csv('dhs_service_records_synthesized_final.csv')

df = dhs_preprocessing(df)
df, service_map = add_service_label(df)
df = add_age_bin(df)

recipient = get_recipient_attribute(df)

## preparing data for association rule mining
* prepare the transaction, i.e., for each recipient, make a list that contains all the services the recipient used

In [2]:
serv_list = []
for groups in df.groupby('id').groups.values():
    serv_list.append(df.loc[groups]['serv'].tolist())

In [3]:
for i in range(10):
    print(serv_list[i])

['S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12']
['S12']
['S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12']
['S12', 'S12', 'S12', 'S12']
['S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12']
['S09', 'S09', 'S09', 'S09', 'S09', 'S09', 'S11', 'S11', 'S11', 'S11', 'S11', 'S11', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12']
['S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12']
['S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12']
['S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12']
['S11', 'S11', 'S11', 'S11', 'S11', 'S11', 'S11', 'S11', 'S11', 'S11', 'S11', 'S11', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12']


In [ ]:
# another way to do the same
serv_list_n = []
for groups in df.groupby('id').groups.values():
    serv_list_n.append(list(df.loc[groups]['serv'].to_numpy()))

# can check equality
serv_list == serv_list_n

In [4]:
for i in range(10):
    print(serv_list[i])

['S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12']
['S12']
['S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12']
['S12', 'S12', 'S12', 'S12']
['S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12']
['S09', 'S09', 'S09', 'S09', 'S09', 'S09', 'S11', 'S11', 'S11', 'S11', 'S11', 'S11', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12']
['S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12']
['S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12']
['S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12']
['S11', 'S11', 'S11', 'S11', 'S11', 'S11', 'S11', 'S11', 'S11', 'S11', 'S11', 'S11', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12', 'S12']


### use mlxtend package

* package has full documentation
* license: permissive BSD, allows for usage, even commercially usable

### in-class work
* read through doc to install
* read through examples

In [5]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.preprocessing import *
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpmax
from mlxtend.frequent_patterns import hmine


In [6]:
# re-do the prep of list of services again
serv_list = []
for groups in df.groupby('id').groups.values():
    serv_list.append(df.loc[groups]['serv'].tolist())

# following the tutorial example 
def oneHotCoding(serv_list):
    te = TransactionEncoder()
    te_ary = te.fit(serv_list).transform(serv_list)
    te_df = pd.DataFrame(te_ary, columns=te.columns_)
    return te_df

serv_oneHot = oneHotCoding(serv_list)

In [7]:
serv_oneHot

,S01,S02,S03,S04,S05,S06,S07,S08,S09,S10,...,S13,S14,S15,S16,S17,S18,S19,S20,S21,S22
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533794,False,False,False,False,False,False,False,False,True,False,...,False,True,False,False,False,False,False,False,False,False
533795,False,False,False,False,False,False,False,False,True,False,...,False,True,False,False,False,False,False,False,False,False
533796,False,False,False,False,False,False,False,False,True,False,...,False,True,False,False,False,False,False,False,False,False
533797,False,False,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False


* using groupby(["id","serv"]) and then oneHotCoding seem to take very long time
* recall we have done something like this earlier, when we transformed df into a matrix for computing correlations
* that code is cleaned up and put in our dhs_util module as a function "get_id_service_matrix"

In [8]:
df_id_serv = get_id_service_matrix(df) # this gives number of times the service is used
df_id_serv.iloc[:,1:23] = df_id_serv.iloc[:,1:23] > 0  # this converts value into True or False


* this was much faster thant the list operation and mlxtend oneHotCoding

In [9]:
df_id_serv

serv,id,S01,S03,S04,S05,S02,S06,S07,S08,S09,...,S13,S14,S15,S16,S17,S18,S19,S20,S21,S22
0,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,3,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,4,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,5,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533794,535604,False,False,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False
533795,535605,False,False,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False
533796,535606,False,False,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False
533797,535607,False,False,False,False,False,True,False,False,True,...,False,False,False,False,False,False,False,False,True,False


* one difference is that this dataframe has the "id" as column, not index
* we could turn it into an index or just use the other columns

In [10]:
apriori(df_id_serv.iloc[:,1:23], use_colnames=True, min_support=0.01)\
    .sort_values(by="support", ascending=False)

,support,itemsets
4,0.941422,(S12)
2,0.153844,(S09)
14,0.139131,"(S12, S09)"
15,0.103528,"(S14, S09)"
6,0.103528,(S14)
25,0.094436,"(S14, S12, S09)"
20,0.094436,"(S14, S12)"
3,0.040882,(S11)
18,0.032106,"(S11, S12)"
10,0.031396,(S19)


* compare

In [11]:
apriori(serv_oneHot, min_support=0.01, use_colnames=True)\
    .sort_values(by="support", ascending=False)

,support,itemsets
4,0.941422,(S12)
2,0.153844,(S09)
14,0.139131,"(S12, S09)"
15,0.103528,"(S14, S09)"
6,0.103528,(S14)
25,0.094436,"(S14, S12, S09)"
20,0.094436,"(S14, S12)"
3,0.040882,(S11)
18,0.032106,"(S11, S12)"
10,0.031396,(S19)


In [12]:
min_freq = 1000 # if we want to set threshold by frequency of the itemsets
min_support = min_freq/serv_oneHot.shape[0]
min_confidence = 0.6
min_rule_support = 0.2
min_lift = 0.15

In [13]:
apriori(serv_oneHot, min_support=min_support,use_colnames=True)\
    .sort_values(by="support", ascending=False)

,support,itemsets
9,0.941422,(S12)
6,0.153844,(S09)
37,0.139131,"(S12, S09)"
11,0.103528,(S14)
39,0.103528,"(S14, S09)"
...,...,...
50,0.001931,"(S11, S18)"
131,0.001930,"(S11, S14, S18, S09)"
110,0.001930,"(S11, S14, S18)"
72,0.001875,"(S02, S14, S09)"


In [14]:
freq_itemset_apriori = apriori(serv_oneHot,min_support=min_support,use_colnames=True)
freq_itemset_apriori.describe()

,support
count,146.000000
mean,0.017443
std,0.080320
min,0.001875
25%,0.002868
50%,0.004436
75%,0.009374
max,0.941422


In [15]:
freq_itemset_fpgrowth = fpgrowth(serv_oneHot,min_support=min_support,use_colnames=True)
freq_itemset_fpgrowth.describe()

,support
count,146.000000
mean,0.017443
std,0.080320
min,0.001875
25%,0.002868
50%,0.004436
75%,0.009374
max,0.941422


In [16]:
freq_itemset_fpmax = fpmax(serv_oneHot,min_support=min_support,use_colnames=True)
freq_itemset_fpmax.describe()

,support
count,22.000000
mean,0.003213
std,0.001915
min,0.001875
25%,0.002171
50%,0.002484
75%,0.003357
max,0.009342


In [17]:
# compute and print the association rules
def serv_rules(freq_itemsets,metrics,threshold):
    asso_rules = association_rules(freq_itemsets, metric=metrics, min_threshold=threshold)
    return asso_rules.sort_values(by='lift', ascending=False)[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

rule_apriori = serv_rules(freq_itemset_apriori,"confidence",0.60)
rule_fpgrowth = serv_rules(freq_itemset_fpgrowth,"confidence",0.60)

In [18]:
rule_fpgrowth

,antecedents,consequents,support,confidence,lift
126,"(S06, S09)","(S21, S12)",0.002486,0.857789,96.948349
125,"(S06, S12)","(S21, S09)",0.002486,0.722767,90.929116
209,"(S02, S09)","(S03, S12)",0.002182,0.969218,78.903093
204,(S02),"(S03, S12)",0.003921,0.935628,76.168573
124,"(S06, S12, S09)",(S21),0.002486,0.987351,74.462712
...,...,...,...,...,...
116,"(S21, S06)",(S12),0.003215,0.701554,0.745206
83,(S17),(S12),0.008674,0.668882,0.710502
102,(S21),(S12),0.008848,0.667279,0.708799
26,"(S19, S09)",(S12),0.003432,0.651494,0.692032


In [19]:
hmine(serv_oneHot,min_support=0.0001,use_colnames=True)


,support,itemsets
0,0.000654,(S01)
1,0.000305,"(S12, S01)"
2,0.004191,(S02)
3,0.004142,"(S02, S03)"
4,0.000146,"(S02, S04, S03)"
...,...,...
930,0.000141,"(S21, S19)"
931,0.002471,(S20)
932,0.000126,"(S20, S21)"
933,0.01326,(S21)


In [20]:
fpmax(serv_oneHot,min_support=0.0001,use_colnames=True)


,support,itemsets
0,0.000199,"(S08, S12)"
1,0.000305,"(S12, S01)"
2,0.000124,"(S22, S12)"
3,0.000133,"(S02, S12, S04, S03)"
4,0.000131,"(S14, S12, S09, S03, S04)"
...,...,...
59,0.000122,"(S14, S21, S12, S09, S18, S11)"
60,0.000210,"(S14, S21, S12, S09, S18, S15)"
61,0.000204,"(S14, S19, S12, S09, S18)"
62,0.000169,"(S14, S19, S12, S09, S15)"


In [21]:
freq_itemset_fpgrowth = fpmax(serv_oneHot,min_support=0.0001,use_colnames=True)
asso_rules = association_rules(freq_itemset_fpgrowth, metric="support", min_threshold=0.0003,support_only=True)
asso_rules.sort_values(by='lift', ascending=False)[['antecedents', 'consequents', 'support', 'confidence', 'lift']]


,antecedents,consequents,support,confidence,lift
0,(S12),(S01),0.000305,NaN,NaN
1,(S01),(S12),0.000305,NaN,NaN
2,"(S14, S17, S12, S09, S18)",(S15),0.000326,NaN,NaN
3,"(S14, S17, S12, S09, S15)",(S18),0.000326,NaN,NaN
4,"(S14, S17, S12, S18, S15)",(S09),0.000326,NaN,NaN
...,...,...,...,...,...
59,(S17),"(S14, S12, S09, S18, S15)",0.000326,NaN,NaN
60,(S12),"(S14, S17, S09, S18, S15)",0.000326,NaN,NaN
61,(S09),"(S14, S17, S12, S18, S15)",0.000326,NaN,NaN
62,(S18),"(S14, S17, S12, S09, S15)",0.000326,NaN,NaN


In [22]:
def predict(antecedent, rules, consequents_only = False):
    # get the rules for this antecedent
    preds = rules[rules['antecedents'] == antecedent]
    if consequents_only:
        # a way to convert a frozen set with one element to string
        preds = preds['consequents'].apply(iter).apply(next)
    return preds

In [23]:
rule_fpmax = association_rules(freq_itemset_fpmax, metric="confidence", min_threshold=0.001, support_only=True)

In [24]:
predict({"S06"}, rule_fpmax, consequents_only=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
38,(S06),"(S21, S12, S09)",NaN,NaN,0.002486,NaN,NaN,NaN,NaN,NaN


In [25]:
predict({"S09"}, rule_fpmax, consequents_only=False)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
5,(S09),"(S02, S14)",NaN,NaN,0.001875,NaN,NaN,NaN,NaN,NaN
19,(S09),"(S03, S02, S12)",NaN,NaN,0.002182,NaN,NaN,NaN,NaN,NaN
25,(S09),"(S10, S12)",NaN,NaN,0.002754,NaN,NaN,NaN,NaN,NaN
39,(S09),"(S21, S06, S12)",NaN,NaN,0.002486,NaN,NaN,NaN,NaN,NaN
67,(S09),"(S07, S11, S14, S12)",NaN,NaN,0.002055,NaN,NaN,NaN,NaN,NaN
97,(S09),"(S14, S18, S12, S16)",NaN,NaN,0.002879,NaN,NaN,NaN,NaN,NaN
113,(S09),"(S13, S14, S12)",NaN,NaN,0.004436,NaN,NaN,NaN,NaN,NaN
129,(S09),"(S14, S15, S17)",NaN,NaN,0.001982,NaN,NaN,NaN,NaN,NaN
143,(S09),"(S15, S17, S12)",NaN,NaN,0.002482,NaN,NaN,NaN,NaN,NaN
157,(S09),"(S14, S17, S12)",NaN,NaN,0.003945,NaN,NaN,NaN,NaN,NaN


In [26]:
predict({"S09"}, rule_fpgrowth)


,antecedents,consequents,support,confidence,lift
15,(S09),"(S14, S12)",0.094436,0.613843,6.500073
9,(S09),(S14),0.103528,0.672938,6.500073
0,(S09),(S12),0.139131,0.904362,0.960634


In [27]:
serv_list =  ['S'+str(i).zfill(2) for i in range(1,23)]
for i in serv_list:
    print(i)
    if (len(predict({i},rule_fpgrowth))>0):
        print(i), print(predict({i},rule_fpgrowth))

S01
S02
S02
    antecedents consequents   support  confidence       lift
204       (S02)  (S03, S12)  0.003921    0.935628  76.168573
200       (S02)       (S03)  0.004142    0.988377  72.213908
201       (S02)       (S12)  0.003970    0.947251   1.006192
S03
S03
   antecedents consequents   support  confidence      lift
16       (S03)       (S12)  0.012284    0.897482  0.953326
S04
S05
S05
    antecedents consequents   support  confidence      lift
175       (S05)       (S12)  0.008057    0.636996  0.676632
S06
S06
    antecedents consequents   support  confidence       lift
115       (S06)       (S21)  0.004582    0.784477  59.162639
S07
S07
    antecedents consequents   support  confidence       lift
141       (S07)  (S11, S12)  0.005105     0.79795  24.853835
137       (S07)       (S11)  0.006398     1.00000  24.460386
138       (S07)       (S12)  0.005105     0.79795   0.847601
S08
S09
S09
   antecedents consequents   support  confidence      lift
15       (S09)  (S14, S12)  0.094

In [28]:
rule_fpgrowth.info()

<class 'pandas.core.frame.DataFrame'>
Index: 212 entries, 126 to 175
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   antecedents  212 non-null    object 
 1   consequents  212 non-null    object 
 2   support      212 non-null    float64
 3   confidence   212 non-null    float64
 4   lift         212 non-null    float64
dtypes: float64(3), object(2)
memory usage: 9.9+ KB


In [29]:
rule_fpgrowth.sort_values("support",ascending=False)


,antecedents,consequents,support,confidence,lift
0,(S09),(S12),0.139131,0.904362,0.960634
8,(S14),(S09),0.103528,1.000000,6.500073
9,(S09),(S14),0.103528,0.672938,6.500073
14,(S14),"(S12, S09)",0.094436,0.912184,6.556292
13,"(S12, S09)",(S14),0.094436,0.678758,6.556292
...,...,...,...,...,...
198,"(S11, S18, S09)",(S14),0.001930,0.999030,9.649879
196,"(S11, S18)",(S14),0.001930,0.999030,9.649879
199,"(S11, S18)","(S14, S09)",0.001930,0.999030,9.649879
210,"(S02, S14)",(S09),0.001875,1.000000,6.500073


In [30]:
df[(df.serv=="S09")].merge(df[(df.serv =="S12")], on = "id").id.nunique()/df.id.nunique()

0.1391310212270911

In [34]:
predict({"S11", "S18"}, rule_fpgrowth)

,antecedents,consequents,support,confidence,lift
196,"(S11, S18)",(S14),0.001930,0.99903,9.649879
199,"(S11, S18)","(S14, S09)",0.001930,0.99903,9.649879
195,"(S11, S18)",(S09),0.001931,1.00000,6.500073
